In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

# PDFファイルを読込
loader = DirectoryLoader('./data/pdf', glob="./*.pdf",   loader_cls=PyPDFLoader)
documents = loader.load()

# 結果の表示
print(documents)


In [ ]:
from langchain_text_splitters import CharacterTextSplitter
import tiktoken

# 言語モデルに合うトークナイザー名を取得
encoding_name = tiktoken.encoding_for_model(MODEL_NAME).name

# テキスト分割を作成
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(encoding_name)

# チャンクに分割
texts = text_splitter.split_documents(documents)

# チャンク数と内容の表示
print("texts_size=", len(texts))
for txt in texts[:3]:
    print(txt)
    print("-" * 50)


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# エンベディングモデルの指定
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# インデックスの構築
db = Chroma.from_documents(texts, embedding_model)


In [ ]:
# Retrieverの作成
retriever = db.as_retriever()

# 検索の実施
results = retriever.invoke("有給休暇の付与日数は？")

# 結果を表示
for result in results:
    print(result.page_content)
    print("-" * 50)

In [ ]:
# 保存先を指定
db = Chroma.from_documents(texts, embedding_model, persist_directory="./chroma_db")

# ストレージから復元
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
prompt = ChatPromptTemplate.from_template("""提供されたコンテキストのみに基づいて次の質問に答えてください:

<コンテキスト>
{context}
</コンテキスト>

Question: {input}""")


In [ ]:
# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = ({"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser())


In [ ]:
# チェーンの実行
response = chain.invoke("有給休暇の付与日数は？")

# 結果を表示
print(response)
